In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from dotenv import load_dotenv
_ = load_dotenv()

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
#TOOL THAT THE AGENT WILL HAVE TO HIS DISPOSAL

tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

In [ ]:
#create the agent state. An annotated list of messages that we want to add
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
class Agent:

    def __init__(self, model, tools, system=""):  #a model to use, tools to use and a system message.
        self.system = system
        graph = StateGraph(AgentState) #CREATE GRAPH passing the state.
        graph.add_node("llm", self.call_openai) #CREAT E NODE LLM ADD TO GRAPH and an action to represent this node.
        graph.add_node("action", self.take_action) #CREATE NODE ACTION ADD TO GRAPH and pass also the function to represent this node.
        graph.add_conditional_edges( #CREATE NODE ACTION ADD TO GRAPH and pass also the function to represent this node.
            "llm", #where di edge start
            self.exists_action,  #function
            {True: "action", False: END} #dictionary. How to map the response of the function. If the function response TRUE go to the action otherwise END
        )
        graph.add_edge("action", "llm") #CREATE an EDGE from action to llm
        graph.set_entry_point("llm") #SET ENTRY POINT FOR THE GRAPH
        self.graph = graph.compile() #COMPILE THE GRAPH
        self.tools = {t.name: t for t in tools} #TOOLS DICTIONARY: mapping the name of the tool to the tool itself
        self.model = model.bind_tools(tools) #BIND THE TOOLS, IS LETTING THE MODEL KNOW HAVING THOSE TOOLS THAT IT CAN CALL

    def exists_action(self, state: AgentState):  #FUNCTION THAT REPRESENT THE EDGE DECISION NODE
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState): #FUNCTION THAT REPRESENT THE llm NODE. (AgentState is passed to all the node so all the function)
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages #CONCATENATE THE SYSTEM MESSAGE (SYSTEM PROMPT) TO THE OTHER MESSAGES (USER PROMPTS)
        message = self.model.invoke(messages) #INVOKE THE MODEL 
        return {'messages': [message]} #RITORNA TUTTO IL PROMPT + ULTIMO MESSAGE (ANSWER DELL' LLM)

    def take_action(self, state: AgentState): #FUNTION THAT REPRESENT THE ACTION NODE
        tool_calls = state['messages'][-1].tool_calls #TAKE LAST MESSAGE TAKE THE ATTRIBUTE OF THE AIMessage (TYPE OF LANGCHAIN) THAT IS A LIST OF TOOLS
        results = []
        for t in tool_calls: #FOR EACH TOOL 
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # IF IT'S IN THE ALLOWED ONE (check for bad tool name from LLM)
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [ ]:
#This allow you to print the draw of the graph.

from IPython.display import Image

Image(abot.graph.get_graph().draw_png())

In [ ]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
#LOG OF THE PREVIOUS GRAPH INVOCATION
#Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_PvPN1v7bHUxOdyn4J2xJhYOX'}
#Back to the model!

In [ ]:
result['messages'][-1].content

In [ ]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
#LOG OF THE PREVIOUS GRAPH INVOCATION
#Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_1SqGYuEtOOFN1yiIHSQTPnvE'}
#Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_8RiM72Y7G8V7c3HEEAML1SKP'}
#Back to the model!

#AS you can see here before to go back to the model it performs 2 actions in the same transaction before to go to the model

In [ ]:
result['messages'][-1].content

In [ ]:

#Calling: {'name': 'tavily_search_results_json', 'args': {'query': '2024 Super Bowl winner'}, 'id': 'call_HBUU1Lo9WSgKCPKYCAStSb7g'}
#Back to the model! 

(needs to come back with the answer of the first action)

#Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Kansas City Chiefs headquarters location'}, 'id': 'call_qMwT4gLkDcvIlJmXDlW4jBll'}
#Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of Missouri 2023'}, 'id': 'call_9lsnrIuDSFpdbEGMnY0VXaAN'}
#Back to the model!

First perform an action and use a tool for the first iteration
Than go back to the model because it needs to know the answer before the next action
After the answer it can perform the next action that is to call 2 times the same tool